In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=2*forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list(np.log(rawArrayDatas[1]))
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list(np.log(rawArrayDatas[1][:-forecastDay] ))
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-3*forecastDay] & np.log
    ds = rawArrayDatas[0][:-3*forecastDay]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-3*forecastDay]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of 2*forecastDay:  rawArrayDatas[1][-3*forecastDay:-forecastDay]
    testY= rawArrayDatas[1][-3*forecastDay:-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
            realForecastDictionary['Bayseian']=Bayseian(txsForRealForecastBayesian,forecastDay,'day')

            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
     
            realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')

            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
            realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    # tf.reset_default_graph()
    # realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay, feature)

    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return realForecastDictionary[nameOfBestAlgorithm]


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
        listedLogPredict=test_predict[-1].tolist()
    return [np.exp(y) for y in listedLogPredict]

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print('\n')
    print('LSTM forecast :',mockForecastDictionary['LSTM'] , '\n@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian forecast :',mockForecastDictionary['Bayseian'], '\n@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print('\n')
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm



In [2]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [3]:
columns=['ds','y']
df=pd.read_table('webMonth.csv', sep=',',header=None,names=columns )
forecastDay=int(len(df)*0.1)
print(len(df), forecastDay)


98 9


In [4]:
# df['y'][-forecastDay:]=[1]*forecastDay
rawArrayDatas=[[i for i in df['ds']],[i for i in df['y']]]

In [6]:
# columns=['ds','y']
# txs=pd.read_table('walMonth_train.csv', sep=',',header=None,names=columns )

In [7]:
# len(txs)
# LSTM(txs, 7, features)
# Bayseian(txs, 7, 'month')

In [8]:
answer=LearningModuleRunner(rawArrayDatas,7,9,'month') #7은 processId

[step: 0] loss: 7.058945655822754
[step: 1] loss: 3.811281204223633
[step: 2] loss: 1.7838832139968872
[step: 3] loss: 0.6835789680480957
[step: 4] loss: 0.24877071380615234
[step: 5] loss: 0.24200588464736938
[step: 6] loss: 0.4317721128463745
[step: 7] loss: 0.6298753619194031
[step: 8] loss: 0.7370294332504272
[step: 9] loss: 0.73935866355896
[step: 10] loss: 0.6657836437225342
[step: 11] loss: 0.5538766980171204
[step: 12] loss: 0.43429720401763916
[step: 13] loss: 0.3259832561016083
[step: 14] loss: 0.23714883625507355
[step: 15] loss: 0.16899918019771576
[step: 16] loss: 0.11951932311058044
[step: 17] loss: 0.08599355816841125
[step: 18] loss: 0.06604373455047607
[step: 19] loss: 0.05761495605111122
[step: 20] loss: 0.05852103978395462
[step: 21] loss: 0.06605970859527588
[step: 22] loss: 0.07700095325708389
[step: 23] loss: 0.08798571676015854
[step: 24] loss: 0.09612330794334412
[step: 25] loss: 0.09948892891407013
[step: 26] loss: 0.09733263403177261
[step: 27] loss: 0.0899850

[step: 244] loss: 0.00017414274043403566
[step: 245] loss: 0.0001740724255796522
[step: 246] loss: 0.00017400190699845552
[step: 247] loss: 0.00017393159214407206
[step: 248] loss: 0.00017386094259563833
[step: 249] loss: 0.0001737899729050696
[step: 250] loss: 0.00017371926514897496
[step: 251] loss: 0.00017364797531627119
[step: 252] loss: 0.0001735776022542268
[step: 253] loss: 0.000173506501596421
[step: 254] loss: 0.00017343484796583652
[step: 255] loss: 0.00017336380551569164
[step: 256] loss: 0.00017329162801615894
[step: 257] loss: 0.00017322083294857293
[step: 258] loss: 0.0001731492520775646
[step: 259] loss: 0.00017307745292782784
[step: 260] loss: 0.00017300579929724336
[step: 261] loss: 0.0001729338546283543
[step: 262] loss: 0.00017286204092670232
[step: 263] loss: 0.00017278979066759348
[step: 264] loss: 0.0001727176713757217
[step: 265] loss: 0.0001726456976030022
[step: 266] loss: 0.0001725732727209106
[step: 267] loss: 0.0001725009351503104
[step: 268] loss: 0.0001724

[step: 474] loss: 0.0001564247941132635
[step: 475] loss: 0.00015634414739906788
[step: 476] loss: 0.0001562640245538205
[step: 477] loss: 0.00015618366887792945
[step: 478] loss: 0.00015610340051352978
[step: 479] loss: 0.00015602345229126513
[step: 480] loss: 0.0001559431984787807
[step: 481] loss: 0.00015586278459522873
[step: 482] loss: 0.00015578274906147271
[step: 483] loss: 0.000155702349729836
[step: 484] loss: 0.0001556223287479952
[step: 485] loss: 0.00015554176934529096
[step: 486] loss: 0.00015546187933068722
[step: 487] loss: 0.0001553815818624571
[step: 488] loss: 0.00015530124073848128
[step: 489] loss: 0.00015522098692599684
[step: 490] loss: 0.00015514127153437585
[step: 491] loss: 0.0001550607557874173
[step: 492] loss: 0.00015498082211706787
[step: 493] loss: 0.0001549001899547875
[step: 494] loss: 0.00015482035814784467
[step: 495] loss: 0.00015474010433536023
[step: 496] loss: 0.00015465989417862147
[step: 497] loss: 0.00015457873814739287
[step: 498] loss: 0.00015

[step: 710] loss: 0.00013789829972665757
[step: 711] loss: 0.00013782294990960509
[step: 712] loss: 0.0001377481676172465
[step: 713] loss: 0.00013767299242317677
[step: 714] loss: 0.00013759805005975068
[step: 715] loss: 0.0001375229039695114
[step: 716] loss: 0.00013744787429459393
[step: 717] loss: 0.00013737328117713332
[step: 718] loss: 0.00013729778584092855
[step: 719] loss: 0.0001372231199638918
[step: 720] loss: 0.00013714810484088957
[step: 721] loss: 0.00013707307516597211
[step: 722] loss: 0.00013699846749659628
[step: 723] loss: 0.00013692420907318592
[step: 724] loss: 0.00013684984878636897
[step: 725] loss: 0.00013677464448846877
[step: 726] loss: 0.00013669989129994065
[step: 727] loss: 0.0001366256328765303
[step: 728] loss: 0.00013655057409778237
[step: 729] loss: 0.00013647685409523547
[step: 730] loss: 0.0001364022318739444
[step: 731] loss: 0.00013632747868541628
[step: 732] loss: 0.00013625359861180186
[step: 733] loss: 0.00013617890363093466
[step: 734] loss: 0.0

[step: 939] loss: 0.00012182358477730304
[step: 940] loss: 0.00012175946176284924
[step: 941] loss: 0.00012169477849965915
[step: 942] loss: 0.0001216302189277485
[step: 943] loss: 0.00012156579759903252
[step: 944] loss: 0.00012150072143413126
[step: 945] loss: 0.00012143670755904168
[step: 946] loss: 0.00012137237354181707
[step: 947] loss: 0.00012130763207096606
[step: 948] loss: 0.00012124377826694399
[step: 949] loss: 0.00012117883306927979
[step: 950] loss: 0.00012111471005482599
[step: 951] loss: 0.00012105047790100798
[step: 952] loss: 0.00012098644947400317
[step: 953] loss: 0.00012092229735571891
[step: 954] loss: 0.00012085798516636714
[step: 955] loss: 0.00012079460429958999
[step: 956] loss: 0.00012073005927959457
[step: 957] loss: 0.00012066659110132605
[step: 958] loss: 0.00012060246081091464
[step: 959] loss: 0.00012053885438945144
[step: 960] loss: 0.00012047457130393013
[step: 961] loss: 0.00012041134323226288
[step: 962] loss: 0.00012034771498292685
[step: 963] loss:

[step: 132] loss: 0.00020335233421064913
[step: 133] loss: 0.00020309702085796744
[step: 134] loss: 0.00020259343727957457
[step: 135] loss: 0.00020199375285301358
[step: 136] loss: 0.00020145898452028632
[step: 137] loss: 0.00020108347234781832
[step: 138] loss: 0.00020088153542019427
[step: 139] loss: 0.00020081002730876207
[step: 140] loss: 0.00020081066759303212
[step: 141] loss: 0.0002008293231483549
[step: 142] loss: 0.00020081222464796156
[step: 143] loss: 0.00020071429025847465
[step: 144] loss: 0.00020051315368618816
[step: 145] loss: 0.0002002297405852005
[step: 146] loss: 0.00019992247689515352
[step: 147] loss: 0.00019965333922300488
[step: 148] loss: 0.0001994618069147691
[step: 149] loss: 0.00019934419833589345
[step: 150] loss: 0.00019926704408135265
[step: 151] loss: 0.00019919234910048544
[step: 152] loss: 0.00019909284310415387
[step: 153] loss: 0.00019896180310752243
[step: 154] loss: 0.00019880390027537942
[step: 155] loss: 0.00019862927729263902
[step: 156] loss: 0

[step: 356] loss: 0.00016771136142779142
[step: 357] loss: 0.00016755354590713978
[step: 358] loss: 0.00016739680722821504
[step: 359] loss: 0.00016723906446713954
[step: 360] loss: 0.00016708180191926658
[step: 361] loss: 0.0001669246848905459
[step: 362] loss: 0.00016676764062140137
[step: 363] loss: 0.00016661059635225683
[step: 364] loss: 0.00016645353753119707
[step: 365] loss: 0.00016629601304885
[step: 366] loss: 0.0001661391870584339
[step: 367] loss: 0.000165982884936966
[step: 368] loss: 0.00016582521493546665
[step: 369] loss: 0.00016566889826208353
[step: 370] loss: 0.00016551197040826082
[step: 371] loss: 0.00016535501345060766
[step: 372] loss: 0.0001651986822253093
[step: 373] loss: 0.00016504230734426528
[step: 374] loss: 0.00016488574328832328
[step: 375] loss: 0.00016472850984428078
[step: 376] loss: 0.00016457214951515198
[step: 377] loss: 0.00016441640036646277
[step: 378] loss: 0.0001642598072066903
[step: 379] loss: 0.00016410378157161176
[step: 380] loss: 0.00016

[step: 584] loss: 0.00013432049308903515
[step: 585] loss: 0.00013418981689028442
[step: 586] loss: 0.00013405820936895907
[step: 587] loss: 0.00013392763503361493
[step: 588] loss: 0.0001337971625616774
[step: 589] loss: 0.0001336665591225028
[step: 590] loss: 0.00013353602844290435
[step: 591] loss: 0.00013340548321139067
[step: 592] loss: 0.00013327618944458663
[step: 593] loss: 0.0001331461826339364
[step: 594] loss: 0.00013301624858286232
[step: 595] loss: 0.0001328869111603126
[step: 596] loss: 0.0001327569189015776
[step: 597] loss: 0.0001326285710092634
[step: 598] loss: 0.00013249872426968068
[step: 599] loss: 0.0001323705946560949
[step: 600] loss: 0.00013224119902588427
[step: 601] loss: 0.00013211283658165485
[step: 602] loss: 0.00013198450324125588
[step: 603] loss: 0.00013185593707021326
[step: 604] loss: 0.00013172783656045794
[step: 605] loss: 0.00013159993977751583
[step: 606] loss: 0.00013147205754648894
[step: 607] loss: 0.0001313448155997321
[step: 608] loss: 0.0001

[step: 816] loss: 0.00010822401236509904
[step: 817] loss: 0.00010813007247634232
[step: 818] loss: 0.00010803703480632976
[step: 819] loss: 0.00010794330592034385
[step: 820] loss: 0.00010784928599605337
[step: 821] loss: 0.00010775651753647253
[step: 822] loss: 0.00010766338527901098
[step: 823] loss: 0.00010757015115814283
[step: 824] loss: 0.00010747773922048509
[step: 825] loss: 0.00010738488344941288
[step: 826] loss: 0.00010729251516750082
[step: 827] loss: 0.0001072005252353847
[step: 828] loss: 0.00010710782953538
[step: 829] loss: 0.00010701594146667048
[step: 830] loss: 0.00010692432260839269
[step: 831] loss: 0.00010683196160243824
[step: 832] loss: 0.00010674078657757491
[step: 833] loss: 0.0001066492623067461
[step: 834] loss: 0.00010655757068889216
[step: 835] loss: 0.00010646650480339304
[step: 836] loss: 0.00010637578088790178
[step: 837] loss: 0.00010628460586303845
[step: 838] loss: 0.00010619407839840278
[step: 839] loss: 0.00010610358731355518
[step: 840] loss: 0.0

In [9]:
len(answer)

9

In [139]:
answer

[2178.676251601592,
 1461.6115387298612,
 2651.7794099982207,
 3007.6696959836563,
 5874.4991051425168,
 3859.3472809028963,
 3301.8477645126191,
 4105.2784993045379,
 11852.520793829079]